In [ ]:
from tensorflow import keras
from keras.datasets import mnist

from keras import Sequential
from keras.layers import Dense, Input, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

Importar y normalizar datos para el aumentado de l dataset con ImageDataGenerator

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Aumentado de datos

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

Topologia de la red

In [ ]:
model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax')) #para MNIST, num clases = 10 (del 0 al 9)

Optimizadores

In [ ]:
# Se pueden probar otros Optimizadores 
#opt=SGD(learning_rate=0.025, momentum=0.9)
#opt = Adagrad(learning_rate=lr)
opt = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Early stopping (modificado para que no acabe prematuramente; min_delta=0.001 como minimo)

In [ ]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3) 

Checkpoint para guardar el mejor modelo

In [ ]:
# Añade un model checkpoint para guardar el mejor modelo
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

Regularizacion del Learning Rate

In [ ]:
# Se pueden probr otros LRS
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.00001)

LRS propio

In [ ]:
from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    if epoch < 5:
        return lr
    else:
        return lr * 0.1

LRS=LearningRateScheduler(scheduler, verbose=1)

Entrenamiento: para early stopping / checkpoint / regularizacion lrs añadir parametro callback a el .fit

In [ ]:
epochs=50
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr,checkpoint])

Si se quiere entrenar para max rendimiento, no emplear conj de validacion, o emplear el test como validacion

Cargar el mejor modelo y evaluarlo

In [ ]:
## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

DROPOUT aleatoriamente elimina neuronas para evitar sobreajuste en training

In [ ]:
model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

Ejemplo para accuracy >= 99%

In [ ]:
## Solución


# Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from keras import Sequential
from keras.layers import Dense, Input, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')



datagen.fit(x_train)

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.00001)

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=50
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
